In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "4"  # Change "<n>" to the index of the GPU you want to use on node

%load_ext autoreload
%autoreload 2

import time
import math
import copy
import torch
from torch import nn, optim
import numpy as np
import awkward as ak
import pandas as pd
import dask
import vector
import particle
import hepunits

import zuko
import torch
from torch import nn, optim
import lightning as L
from lightning.pytorch import loggers as pl_loggers

from torch.utils.data import DataLoader

from memflow.dataset.data import ParquetData # Class to handle Parquet files
from memflow.dataset.dataset import CombinedDataset
#from memflow.HH.ttH import ttHHardDataset, ttHRecoDataset
from memflow.dataset.tth import ttHHardDataset, ttHRecoDataset
from memflow.callbacks.transfer_flow_callbacks import *
from memflow.models.custom_flows import *
vector.register_awkward() # Enables vector operations on awkward arrays

from matplotlib import pyplot as plt

plt.rcParams.update({'figure.max_open_warning': 100})

vector.register_awkward()

print (f"Running on GPU : {torch.cuda.is_available()}")
accelerator = 'gpu' if torch.cuda.is_available() else 'cpu'
print (f"Accelerator : {accelerator}")
torch.set_float32_matmul_precision('medium')  
if accelerator =='gpu':
    torch.cuda.empty_cache()
    print (torch.cuda.memory_summary(device=None, abbreviated=True))

Running on GPU : True
Accelerator : gpu
|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |      0 B   |      0 B   |      0 B   |      0 B   |
|---------------------------------------------------------------------------|
| Active memory         |      0 B   |      0 B   |      0 B   |      0 B   |
|---------------------------------------------------------------------------|
| Requested memory      |      0 B   |      0 B   |      0 B   |      0 B   |
|-----------------------

# Dataset preparation

In this section, we will load and prepare the datasets for feeding through the transfer model.

Some of the steps of making particles and selecting events is already done in the classes
- HHbbWWDoubleLeptonHardDataset
- HHbbWWDoubleLeptonRecoDataset

We will first get the gen and reco data

In [2]:
data_hard = ParquetData(
    files = [
        '/cephfs/dice/users/sa21722/datasets/MEM_data/ttH/TF_v6/hard/2018/ttH/ttH_HToInvisible_M125.parquet',
    ], # these are the files we want to load, there can be several
    # in parquet data there is no tree
    lazy = True, # Explained above.
    N = 100000, # this is to load only the N first events in the tree, 
    # in case you are just playing/debugging and don't need to load all the data (can be slow)
    # to load all, just comment out
)

print(data_hard)

Data object
Loaded branches:
   ... file: 100000
   ... sample: 100000
   ... tree: 100000
Branch in files not loaded:
   ... Generator_scalePDF
   ... Generator_weight
   ... Generator_x1
   ... Generator_x2
   ... Generator_xpdf1
   ... Generator_xpdf2
   ... W_minus_from_antitop_eta
   ... W_minus_from_antitop_genPartIdxMother
   ... W_minus_from_antitop_idx
   ... W_minus_from_antitop_mass
   ... W_minus_from_antitop_pdgId
   ... W_minus_from_antitop_phi
   ... W_minus_from_antitop_pt
   ... W_minus_from_antitop_status
   ... W_minus_from_antitop_statusFlags
   ... W_plus_from_top_eta
   ... W_plus_from_top_genPartIdxMother
   ... W_plus_from_top_idx
   ... W_plus_from_top_mass
   ... W_plus_from_top_pdgId
   ... W_plus_from_top_phi
   ... W_plus_from_top_pt
   ... W_plus_from_top_status
   ... W_plus_from_top_statusFlags
   ... Z_from_higgs_eta
   ... Z_from_higgs_genPartIdxMother
   ... Z_from_higgs_idx
   ... Z_from_higgs_mass
   ... Z_from_higgs_pdgId
   ... Z_from_higgs_phi
  

In [3]:
data_reco = ParquetData(
    files = [
        '/cephfs/dice/users/sa21722/datasets/MEM_data/ttH/TF_v6/reco/2018/ttH/ttH_HToInvisible_M125.parquet',
    ],
    lazy = True, # Explained above.
    N = data_hard.N,
)
print(data_reco)

Data object
Loaded branches:
   ... file: 100000
   ... sample: 100000
   ... tree: 100000
Branch in files not loaded:
   ... Generator_scalePDF
   ... Generator_weight
   ... Generator_x1
   ... Generator_x2
   ... Generator_xpdf1
   ... Generator_xpdf2
   ... InputMet_phi
   ... InputMet_pt
   ... cleanedJet_btagDeepFlavB
   ... cleanedJet_eta
   ... cleanedJet_mass
   ... cleanedJet_phi
   ... cleanedJet_pt
   ... event
   ... ncleanedBJet
   ... ncleanedJet
   ... region
   ... weight_nominal
   ... xs_weight


Apply Mask

Use mask to cut our events, only keeping the ones passing our selection

In [4]:
mask_hard = np.logical_and.reduce( # All conditions must be True for an event to pass the mask
    # Each element inside the list below is a condition that needs to be met
    [
        # Higgs decay : H->ZZ->4nu #
        ak.num(data_hard['higgs_idx']) == 1, # Events where there is exactly 1 Higgs particle
        ak.num(data_hard['Z_from_higgs_idx']) == 2, # 2 Z bosons from Higgs
        ak.num(data_hard['neutrinos_from_Z_idx']) == 4, # 2 neutrinos from each Z boson
        # top decay : t->b q qbar #
        ak.num(data_hard['top_idx']) == 1,
        ak.num(data_hard['W_plus_from_top_idx']) == 1,
        ak.num(data_hard['quark_from_W_plus_idx']) == 1,
        ak.num(data_hard['antiquark_from_W_plus_idx']) == 1,
        # antitop decay : tbar->bbar q qbar #
        ak.num(data_hard['antitop_idx']) == 1,
        ak.num(data_hard['W_minus_from_antitop_idx']) == 1,
        ak.num(data_hard['quark_from_W_minus_idx']) == 1,
        ak.num(data_hard['antiquark_from_W_minus_idx']) == 1,
    ]
)
print (f'Selecting {mask_hard.sum()} events out of {len(mask_hard)}')
print (f'Before cut : {data_hard.events}')

data_hard.cut(mask_hard) # Apply the mask

print (f'After cut : {data_hard.events}')

Selecting 39842 events out of 100000
Before cut : 100000
After cut : 39842


In [5]:
mask_reco = data_reco['region'] == 0 # 0 is signal region
print('Before cut: ', data_reco.events, 'events')
data_reco.cut(mask_reco)
print('After cut: ', data_reco.events, 'events')

Before cut:  100000 events
After cut:  49446 events


In [6]:
# Initialize HardDataset and RecoDataset for ttH process
hard_dataset = ttHHardDataset(
    data = data_hard,
    selection=[     # Specify particles to be fed to ML model
        'higgs',
        'tops',
        'bottoms',
        'Ws',
        'quarks',
        'Zs',
        'neutrinos'
    ],
    coordinates = 'cylindrical', # Choose 'cylindrical' for [pt, eta, phi, m] or 'cartesian' for [px, py, pz, E]
    apply_boost = False,  # Set whether or not to boost to the center of mass (CM) frame
    apply_preprocessing = True,
    build = True,  # Re-save processed data to the specified directory
    dtype = torch.float32,
)

print(hard_dataset)

FieldNotFoundError: no field 'generator_info' in record with 160 fields

This error occurred while attempting to slice

    <Array-typetracer [...] type='## * {event: ?uint64, Generator_weight: ?...'>

with

    'generator_info'

In [ ]:
reco_dataset = ttHRecoDataset(
    data = data_reco,
    selection=[     # Include reconstructed particles: jets and MET
        'jets',
        'met'
    ],
    coordinates = 'cylindrical',
    apply_boost = False,
    apply_preprocessing = True,
    build = True,
    dtype = torch.float32,
)


Just to understand a bit better what is in our dataset, we can plot the different quantities for each partycle/type.

We can plot both the raw values, and the ones after pre-processing.

About preprocessing, for several reasons we do not want to feed the raw values to the ML model. 
So we apply different preprocessing schemes (apply a log, standard scaling, min-max scaling, etc), but we still want to check their effect.

In [ ]:
print ('Hard-level')
print ('Before preprocessing')

hard_dataset.plot(selection=True, raw=True)
print ('After preprocessing')
hard_dataset.plot(selection=True,raw=False)

print ('Reco-level')
print ('Before preprocessing')
reco_dataset.plot(selection=True,raw=True)
print ('After preprocessing')
reco_dataset.plot(selection=True,raw=False)

Now, we want to combine the reco and gen information.

To avoid doing it by hand, a combined dataset has been created. 

In [ ]:
comb_dataset = CombinedDataset(
    hard_dataset = hard_dataset,
    reco_dataset = reco_dataset,
    intersection_branch = 'event',
)
print (comb_dataset)

In [ ]:
# To check what is contained in all our batches, we make a classic pytorch loader, and print the content
loader_comb = DataLoader(
    comb_dataset,
    batch_size = 256,
)
batch = next(iter(loader_comb))

print ('Reco')
for obj,mask,sel in zip(batch['reco']['data'],batch['reco']['mask'],reco_dataset.selection):
    print ('\t',sel,obj.shape,mask.shape)
print ('Hard')
for obj,mask,sel in zip(batch['hard']['data'],batch['hard']['mask'],hard_dataset.selection):
    print ('\t',sel,obj.shape,mask.shape)


The batches that we provide to the loader contain the different types of particles for reco and gen level.
The shape is `[N,S,F]`
- `N` : number of events in the batch (this is the batch size that you decide yourself)
- `S` : sequence length = number of particles (eg, number of electrons, or jets) (see explanation of mask below)
- `F` : number of features (eg, pt, eta, pdgid, btag score, etc) for each particle

For each type of particle we provide the data (actual values in the training) with shape above, but also the mask.
This mask comes from the fact that we need to feed a fixed-size tensor to the ML model, but we do not have fixed size inputs (number of jets can vary for example). So we "pad" the tensors with 0s, but the fact that these particles are missing are saved in the mask that has False's. In the transformer this will be used in the self-attention. The mask has a shape `[N,S]`.

Now we need to split the dataset into a training and validation dataset, and make them into dataloaders.

In [ ]:
# Split dataset into training and validation
# Not randomly for reproducilibility, but just based on number

train_frac = 0.9
indices = torch.arange(len(comb_dataset))
sep = int(train_frac*len(comb_dataset))
train_indices = indices[:sep]
valid_indices = indices[sep:]

comb_dataset_train = torch.utils.data.Subset(comb_dataset,train_indices)
comb_dataset_valid = torch.utils.data.Subset(comb_dataset,valid_indices)

print (f'There will be {len(comb_dataset_train)} events in the training dataset, and {len(comb_dataset_valid)} in the validation set')

batch_size = 1024

loader_comb_train = DataLoader(
    comb_dataset_train,
    batch_size = batch_size,
    shuffle = True,
)
loader_comb_valid = DataLoader(
    comb_dataset_valid,
    batch_size = 10000,
    shuffle = False,
)

# Creating the model

A specific pytorch class has been coded for the transfer flow

Most of these arguments would require a lot of text, so will give some cues but can explain later

In [ ]:
from memflow.models.transfer_flow_model import TransferFlow

model = TransferFlow(
    # Embedding arguments #
    embed_dims = [32,64], # number of neurons per layer to embed the particle components to higher dimension
    embed_act = nn.GELU,
    # Particle features, names, masks, and number for printouts and logging #
    n_hard_particles_per_type = hard_dataset.number_particles_per_type,
    hard_particle_type_names = hard_dataset.selection,
    hard_input_features_per_type = hard_dataset.input_features,
    n_reco_particles_per_type = reco_dataset.number_particles_per_type,
    reco_particle_type_names = reco_dataset.selection,
    reco_input_features_per_type = reco_dataset.input_features,
    flow_input_features = [ # features to be used in the flows (different from the tranformer)
        ['pt','phi'],       # met
        ['pt','eta','phi'], # jets
    ],
    reco_mask_attn = reco_dataset.attention_mask,
    hard_mask_attn = hard_dataset.attention_mask,
    # Transformer arguments #
    onehot_encoding = True, # add onehot encoded position vector to features
    transformer_args = { # to be passed to the Transformer pytorch class
        'nhead' : 8,
        'num_encoder_layers' : 8, 
        'num_decoder_layers' : 8, 
        'dim_feedforward' : 256, 
        'dropout' : 0., 
        'activation' : 'gelu', 
    },
    # Flow args #
    flow_common_args = { # common args for all flows
        'bins' : 32,
        'transforms' : 5,
        'randperm' : True,
        'passes' : None,
        'hidden_features' : [128] * 3,   
    },
    flow_classes = { # classes for each feature
        'pt'  : zuko.flows.NSF,
        'eta' : UniformNSF,
        'phi' : UniformNCSF,
        'pt'  : zuko.flows.NSF,
    },
    flow_specific_args = { # specific args for each class above
        'eta' : {'bound' : 1.},
        'phi' : {'bound' : 1.},
    },
    flow_mode = 'global', # 'global', 'type' or 'particle'
)
model = model.cpu()

# Below are some testing of individual workings of the model
# This is just to make sure there is no bug before running the training loop

# Testing the model #
# We get a single batch from the loader
batch = next(iter(loader_comb))
# This returns the -log(prob), the combined mask and weights
# They all have shape [N,S], as we have a log prob per particle
log_probs, mask, weights = model(batch)
print ('log_probs',log_probs,log_probs.shape)
print ('mask',mask,mask.shape)
print ('weights',weights,weights.shape)

# Separately test to get the total log prob #
log_probs_tot = model.shared_eval(batch,0,'test')
print ('tot log probs',log_probs_tot)

# Separately test sampling the model #
samples = model.sample(batch['hard']['data'],batch['hard']['mask'],batch['reco']['data'],batch['reco']['mask'],N=100)
print ('samples')
for sample in samples:
    print ('\t',sample.shape)

print (model)


# Training the model

We use [lightning](https://lightning.ai/docs/pytorch/stable/) to remove some boilerplate code, mostly during training.

The training is done through a trainer (see below). To log all the interesting quantities (and plots, will do that later), we use [comet](https://www.comet.com) (you will probably have to create an account, or link your github account).

This tool is pretty cool to plot the different metrics, I can pass to you a setup later. If you want to avoid this, comment out the `logger` part.

All the following code are tools for the training, will dedicate a discussion later on this.

In [ ]:
##### Parameters #####
epochs = 10 # use small number for testing here, in practice would use in the hundreds
steps_per_epoch_train = math.ceil(len(comb_dataset_train)/loader_comb_train.batch_size)

print (f'Training   : Batch size = {loader_comb_train.batch_size} => {steps_per_epoch_train} steps per epoch')

##### Optimizer #####
optimizer = optim.RAdam(model.parameters(), lr=1e-3)
model.set_optimizer(optimizer)

##### Scheduler #####
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer = optimizer,
    mode='min', 
    factor=0.1, 
    patience=10, 
    threshold=0.001, 
    threshold_mode='rel', 
    cooldown=5, 
    min_lr=1e-7
)
model.set_scheduler_config(
    {
        'scheduler' : scheduler,
        'interval' : 'step' if isinstance(scheduler,optim.lr_scheduler.OneCycleLR) else 'epoch',
        'frequency' : 1,
        'monitor' : 'val/loss',
        'strict' : True,
        'name' : 'scheduler',
    }
)

##### Callbacks #####
callbacks = [
    L.pytorch.callbacks.LearningRateMonitor(logging_interval = 'epoch'),
    L.pytorch.callbacks.ModelSummary(max_depth=2),
] 

# ##### Logger #####
# logger = pl_loggers.CometLogger(
#     save_dir = '../comet_logs',
#     project_name = 'mem-flow-HH',
#     experiment_name = 'HH',
#     offline = False,
# ) 
# logger.log_graph(model)
# logger.experiment.log_notebook(filename=globals()['__session__'],overwrite=True)

##### Trainer #####
trainer = L.Trainer(    
    min_epochs = 5,
    max_epochs = epochs,
    callbacks = callbacks,
    devices = 'auto',
    accelerator = accelerator,
    #logger = logger,
    log_every_n_steps = steps_per_epoch_train,
)
##### Fit #####
trainer.fit(
    model = model, 
    train_dataloaders = loader_comb_train,
    val_dataloaders = loader_comb_valid,
)

# Evaluating the model

We want to make sure the model is doing a good job.

Because our model is used to compute the log probability, it can be a bit hard to represent. Instead what we can use is a sampling of this pdf through sampling with the flow. 

We will plot two cases
- Sampling per event : we select one event specifically, and sample N times and show the distribution
- Sampling bias : for all events, we sample N times, and investigate the bias

We will show both cases below.

Note : here we do plots post training, but these classes below are made to be used as callbacks. This means they can be called automatically during training (typially at the end of the validation for each epoch). The resulting plots will then be logged to comet and be saved online.

Here we will just use them to plot after the training, but you can pass the class instances in the training callbacks above.

## Per-event sampling

We want to see what the pdf looks like for event.

To do that we sample N times for a specific event, and compare the sampled values to the true ones.

In [ ]:
model = model.cuda() # we make sure model is on GPU (to make it faster)
sampling = SamplingCallback(
    dataset = comb_dataset,
    idx_to_monitor = torch.arange(5), # show the first five events (you can use specific events you want to look at)
    N_sample = 100000,  # number of samples per event (this should be high, but also takes longer)
    frequency = 10, # callback parameter : frequency of epochs to record (1 = all epochs)
    raw = True, # if True, will undo the preprocessing
    bins = 101, # bins for the plot
    log_scale = True, # log scale for the plot
)
figures = sampling.make_sampling_plots(model,show=True)


## Bias plotting

We want to make sure the sampling (and therefore the pdf) is not biased.

To do so, we sample N times for each event, and compute the different sample-true.

This will be plotted in 1D, as a function of the true value, and then estimating the bias coverage.

In [ ]:
model = model.cuda() # we make sure model is on GPU (to make it faster)
bias = BiasCallback(
    dataset = comb_dataset,
    N_sample = 50, # if very high, will be very slow
    batch_size = 10000, # compute certain number of events at a time (all events would break RAM)
    # N_batch = 1, # Limits the number of batches to process (eg =1 to make it faster), comment out to use all of them
    frequency = 10, # callback parameter : frequency of epochs to record (1 = all epochs)
    raw = True, # if True, will undo the preprocessing
    bins = 101, # plot bins
    points = 20, # number of points for the bias plot
    log_scale = True, # log scale for the plot
)
figs = bias.make_bias_plots(model,show=True)